# Importing/Install Libraries

In [ ]:
# Cell 1: Install necessary packages if not already installed
try:
    import cv2
except ImportError:
    !pip install opencv-python opencv-python-headless
    import cv2

  Using cached opencv-python-4.10.0.84.tar.gz (95.1 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached opencv-python-headless-4.10.0.84.tar.gz (95.1 MB)


In [ ]:
try:
    import numpy as np
except ImportError:
    !pip install numpy
    import numpy as np

In [ ]:
try:
    import matplotlib.pyplot as plt
except ImportError:
    !pip install matplotlib
    import matplotlib.pyplot as plt

In [ ]:
import os

# Loading and Processing Videos

In [ ]:
# Path to the dataset
dataset_path = 'dataset/'

# List of video files
video_files = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path) if f.endswith('.mp4')]

# Display the list of video files
print("Video files:", video_files)


# Loading the Pretrained Model

In [ ]:
# Load YOLOv3 model and weights
yolo_net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# Load the COCO names file
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = yolo_net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in yolo_net.getUnconnectedOutLayers()]

# Display the loaded classes
print("Loaded classes:", classes)


# Detecting Signals in Videos

In [ ]:
def detect_signals(frame, net, output_layers, classes):
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    result = []
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            result.append((label, confidence, (x, y, w, h)))
    
    return result

# Process each video and detect signals
for video_file in video_files:
    cap = cv2.VideoCapture(video_file)
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        if frame_count % 30 == 0:  # Process every 30th frame
            detections = detect_signals(frame, yolo_net, output_layers, classes)
            
            for (label, confidence, (x, y, w, h)) in detections:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            # Display the frame with detections
            cv2.imshow("Frame", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    cap.release()
cv2.destroyAllWindows()


# Evaluation and Problems

## Problems Encountered
1. **False Positives and Negatives**: The model may detect non-signal objects as signals (false positives) or miss detecting actual signals (false negatives).
2. **Lighting and Weather Conditions**: Variations in lighting and weather conditions can affect the detection accuracy.
3. **Occlusions**: Signals that are partially obscured by other objects may not be detected.
4. **Model Limitations**: The pre-trained model might not be trained specifically for traffic signals, leading to reduced accuracy.

## System Capability
The system can detect signals to a certain extent, but it may not be able to detect all signals accurately due to the aforementioned problems. Improving the training data and using models specifically trained on traffic signals could enhance the detection performance.

## Why the System Might Fail
1. **Training Data**: The model may not have been trained on a diverse enough dataset of traffic signals.
2. **Real-Time Processing**: Processing videos in real-time can be computationally intensive and might lead to missed detections if the system cannot keep up with the video frame rate.
3. **Environmental Factors**: Variations in the environment, such as shadows, reflections, and weather conditions, can adversely affect the detection accuracy.
